# CSE204 - Project : Relationship prediction through SMS chat analysis


gde sam ovo nasla: https://medium.com/analytics-vidhya/text-based-communication-analysis-with-machine-learning-17138c0a4a4e
nltk tutorijal: https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk

In [ ]:
#ovo nam moze pomoci ako hocemo da analiziramo vreme slanja i ucestalost poruka
df_text=pd.read_csv('text_sample.csv')
df_text.info()
#treba da vidimo da odlucimo koje kolone hocemo za nase podatke, ovo je primer: 
"""<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62513 entries, 0 to 62512
Data columns (total 7 columns):
message_type    62513 non-null object
text            62513 non-null object
time            62513 non-null object
ID              62513 non-null object
group_list      62513 non-null object
term            62513 non-null object
sent_by         62513 non-null object
dtypes: object(7)
memory usage: 3.3+ MB"""

#Extracting year and month from 'time' column and append the dataframe with new columns.
df_text['year'] = pd.DatetimeIndex(df_text['time']).year
df_text['month'] = pd.DatetimeIndex(df_text['time']).month
df_text.head()



In [ ]:
#ovako se broji ko je poslao koliko poruka mesecno, moze da bude korisno za jednostrana muvanja :) 

# Getting the message count by message type and month
# reset_index() gives a column for counting, after groupby uses year and category
df_cnt = (df_text.reset_index()
          .groupby(['month','message_type'], as_index=False)
          .count()
          # rename isn't strictly necessary here, it's just for readability
          .rename(columns={'index':'count'})
       )
#sorting the values by month
df_cnt.sort_values(by = 'month', ascending = True)

In [ ]:
#ciscenje podataka od nebitnih stvari pomocu ovog nlkta, ovo treba proveriti da li nam stvarno treba

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords
set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
from contractions import contractions_dict
import unicodedata

def remove_punctuation(text):
    text = ''.join([i for i in text if not i.isdigit()])
    return re.sub("[!@#$+%*:()/|,;:'-]", ' ', text)

def removebrackets(text):
    return re.sub('[\(\[].*?[\)\]]', ' ', text)

def remove_accented_chars(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

def remove_special_chars(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    return re.sub(pattern, '', text)

def remove_stopwords(text):
   # text_tokens = word_tokenize(text)
   # text = remove_stopwords(text)
      
    from gensim.parsing.preprocessing import STOPWORDS
    all_stopwords_gensim = STOPWORDS.union(set(['thank', 'need', 'yes', 'okay']))
    text_tokens = word_tokenize(text)
    tokens_without_sw = ' '.join([word for word in text_tokens if not word in all_stopwords_gensim])
    
    return tokens_without_sw

def stemming (text):
    
    ps = nltk.porter.PorterStemmer()
    return ' '.join([ps.stem(word) for word in text.split()])

    stopword_list = stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    return ' '.join([token for token in tokens if token not in stopword_list])

def lemmatize(text):
    text = nlp(text)
    return ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])

def expand_contractions(text, contraction_mapping=contractions_dict):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
def expand_match(contraction):
    match = contraction.group(0)
    first_char = match[0]
    expanded_contraction = contraction_mapping.get(match)\
                            if contraction_mapping.get(match)\
                            else contraction_mapping.get(match.lower())                       
    expanded_contraction = first_char+expanded_contraction[1:]
    return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    return re.sub("'", "", expanded_text)

In [ ]:
#ovo pravi worldcloud, nacin da se lepo predstave najucestalije reci, nzm da li nam stvarno treba ali neka ga ovde

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
wc = WordCloud(stopwords=STOPWORDS,max_font_size=200, max_words=1000000, background_color="white", width=1000, height=1000).generate(' '.join(df_filtered['text_clean']))
plt.figure(figsize=(20,20))
plt.imshow(wc)
plt.axis("off")
plt.show()

In [ ]:
#ovo nam sluzi da izaberemo jos reci koje hocemo da izbacimo 

def clean_text_again(text):
from gensim.parsing.preprocessing import STOPWORDS
all_stopwords_gensim = STOPWORDS.union(set(['thank', 'need', 'yes', 'okay','ok','thanks','morning','hello','sure','hi', 'know', 'got','yesterday']))
text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in all_stopwords_gensim]
return ' '.join([word for word in text_tokens if not word in all_stopwords_gensim])

In [ ]:
#ovo je korisno, sluzi da prepoznamo sintagme reci koje se cesto pojavljuju zajedno

def plot_top_ngrams_barchart(text, n=3): #n is the number of immediate words you need.
      
    new= text.str.split()
    new=new.values.tolist()
    corpus=[word for i in new for word in i]
def _get_top_ngram(corpus, n=None):
        vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) 
                      for word, idx in vec.vocabulary_.items()]
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:10]
top_n_bigrams=_get_top_ngram(text,n)[:10]
    x,y=map(list,zip(*top_n_bigrams))
    sns.barplot(x=y,y=x)

In [ ]:
# Sentiment analysis, ovde zapravo pravimo prediction odnosa na osnovu tokena

#ovo prvo samo plotuje koliko ima "negativnih" a koliko "pozitivnih" interakcija, mi treba da imamo vise klasa od te dve
from textblob import TextBlob
    
def plot_polarity_histogram(text):
    
    def polarity(text):
        return TextBlob(text).sentiment.polarity
    
    df_x = pd.DataFrame() 
    polarity_score =text.apply(lambda x : polarity(x))
    df_filtered['polarity_score']=df_filtered['text'].\
       apply(lambda x : polarity(x))
    polarity_score.hist()

In [ ]:
#ovo je slicno, imamo positive, negative i neutral klase

from textblob import TextBlob
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
def sentiment_vader(text, sid):
    ss = sid.polarity_scores(text)
    ss.pop('compound')
    return max(ss, key=ss.get)
def sentiment_textblob(text):
        x = TextBlob(text).sentiment.polarity
        
        if x<0:
            return 'Negative'
        elif x==0:
            return 'Neutral'
        else:
            return 'Positive'
        
def sentiment_x(x):
    if x<0:
        return 'Negative'
    elif x==0:
        return 'Neutral'
    else:
        return 'Positive' 
        
def plot_sentiment_barchart(text, method):
    if method == 'TextBlob':
        sentiment = text.map(lambda x: sentiment_textblob(x))
    elif method == 'Vader':
        nltk.download('vader_lexicon')
        sid = SentimentIntensityAnalyzer()
        sentiment = text.map(lambda x: sentiment_vader(x, sid=sid))
    else:
        raise ValueError('Textblob or Vader')
    
    df_filtered['polarity']=df_filtered['polarity_score'].\
       apply(lambda x : sentiment_x(x))
  
    plt.figure(figsize=(5, 2), dpi=100)
    plt.bar(sentiment.value_counts().index,
            sentiment.value_counts())